# Kafka Security Event Viewer

This notebook connects to Kafka and displays security events in real-time.

## Prerequisites
- Kafka must be running (`kafka.cert-lab.local:9092`)
- The `security-events` topic must exist

In [ ]:
import os
import json
from datetime import datetime
from kafka import KafkaConsumer, KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
import pandas as pd
from IPython.display import display, clear_output
import time

In [ ]:
# Configuration
KAFKA_SERVERS = os.getenv('KAFKA_BOOTSTRAP_SERVERS', 'kafka.cert-lab.local:9092')
KAFKA_TOPIC = os.getenv('KAFKA_TOPIC', 'security-events')

print(f"Kafka Servers: {KAFKA_SERVERS}")
print(f"Topic: {KAFKA_TOPIC}")

## Check Kafka Connection

In [ ]:
try:
    admin = KafkaAdminClient(bootstrap_servers=KAFKA_SERVERS)
    topics = admin.list_topics()
    print(f"Connected to Kafka!")
    print(f"Available topics: {topics}")
    admin.close()
except Exception as e:
    print(f"Failed to connect: {e}")

## View Recent Events

Read the last N events from the topic.

In [ ]:
def get_recent_events(n=10):
    """Get the most recent N events from Kafka."""
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_SERVERS,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        consumer_timeout_ms=5000,
        value_deserializer=lambda m: json.loads(m.decode('utf-8'))
    )
    
    events = []
    for message in consumer:
        events.append(message.value)
    
    consumer.close()
    return events[-n:] if len(events) > n else events

# Get recent events
events = get_recent_events(20)
print(f"Found {len(events)} events")

In [ ]:
if events:
    # Convert to DataFrame for nice display
    df = pd.DataFrame(events)
    
    # Select key columns if they exist
    columns = ['timestamp', 'event_type', 'device_fqdn', 'severity', 'description', 'certificate_serial']
    available_cols = [c for c in columns if c in df.columns]
    
    display(df[available_cols] if available_cols else df)
else:
    print("No events found. Trigger some events using the Mock EDR!")

## Live Event Stream

Watch events in real-time. **Run this cell and trigger events from another terminal.**

To trigger a test event:
```bash
curl -X POST http://localhost:8082/trigger \
  -H 'Content-Type: application/json' \
  -d '{"device_id": "test-device", "scenario": "Certificate Private Key Compromise", "severity": "critical"}'
```

In [ ]:
def stream_events(duration_seconds=60):
    """Stream events for a specified duration."""
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_SERVERS,
        auto_offset_reset='latest',
        enable_auto_commit=True,
        group_id='jupyter-viewer',
        consumer_timeout_ms=1000,
        value_deserializer=lambda m: json.loads(m.decode('utf-8'))
    )
    
    print(f"Listening for events for {duration_seconds} seconds...")
    print("Trigger events from Mock EDR to see them here.\n")
    print("-" * 80)
    
    start_time = time.time()
    event_count = 0
    
    while time.time() - start_time < duration_seconds:
        try:
            for message in consumer:
                event = message.value
                event_count += 1
                
                print(f"[{event.get('timestamp', 'N/A')}]")
                print(f"  Type: {event.get('event_type', 'unknown')}")
                print(f"  Device: {event.get('device_fqdn', 'unknown')}")
                print(f"  Severity: {event.get('severity', 'unknown')}")
                print(f"  Description: {event.get('description', 'N/A')[:60]}...")
                if event.get('certificate_serial'):
                    print(f"  Certificate: {event.get('certificate_serial')}")
                print("-" * 80)
                
        except StopIteration:
            pass
    
    consumer.close()
    print(f"\nFinished. Received {event_count} events.")

# Stream for 60 seconds (interrupt kernel to stop early)
stream_events(60)

## Event Statistics

In [ ]:
# Get all events for statistics
all_events = get_recent_events(1000)

if all_events:
    df = pd.DataFrame(all_events)
    
    print(f"Total Events: {len(df)}")
    print("\nEvents by Type:")
    if 'event_type' in df.columns:
        print(df['event_type'].value_counts().to_string())
    
    print("\nEvents by Severity:")
    if 'severity' in df.columns:
        print(df['severity'].value_counts().to_string())
    
    print("\nEvents by Device:")
    if 'device_fqdn' in df.columns:
        print(df['device_fqdn'].value_counts().head(10).to_string())
else:
    print("No events to analyze.")